In [1]:
!pip3 install vncorenlp
!pip3 install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=5002caaac1536746f22424e1dc8172626af790711ea404c5b52fac267210fbef
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


In [2]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()        

  Preparing metadata (setup.py) ... - done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=e23b1b73781d2de58446bf31ca8a7101f9ba127b3ca62109c8b1e25f5fe7e1ef
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  9% |  2% |


In [3]:
# Import libraries
from vncorenlp import VnCoreNLP
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
from tqdm import tqdm
import pickle
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import os
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW, RobertaTokenizer, RobertaTokenizerFast, RobertaModel, AutoTokenizer
from datetime import datetime
import glob

In [4]:
def changeToOther(topic=''):
    top_9_subtopic = csv['Topic'].value_counts()[:9].index.tolist()
    if topic not in top_9_subtopic: return 'Other'
    return topic

In [5]:
import pandas as pd
csv = pd.read_csv('/kaggle/input/vne-sub-topic-1/Vne_sub_topic_1.csv')
csv['Topic'] = csv['Topic'].apply(lambda x: changeToOther(x))
csv = csv.sample(frac=1)
csv[csv['Topic'] == 'Other'] = csv[csv['Topic'] == 'Other'][:500]
csv = csv.dropna()
csv['Topic'].value_counts()

Other                      500
the-thao_bong-da           300
giao-duc_tin-tuc           296
the-thao_cac-mon-khac      292
suc-khoe_tin-tuc           292
giai-tri_gioi-sao          271
du-lich_diem-den           264
the-thao_world-cup-2022    259
khoa-hoc_tin-tuc           253
suc-khoe_cac-benh          246
Name: Topic, dtype: int64

In [6]:
dataset = csv[['Content', 'Topic']]
dataset = dataset.sample(frac=1)
data = dataset['Content']
label = dataset['Topic']

In [7]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/kaggle/input/vncorenlp/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [8]:
import string
import re
# Text preprocessing 
def text_preprocessing(text=''):
  text = text.lower()
  text = text.translate(text.maketrans(string.punctuation, ' '*len(string.punctuation)))
  text = re.sub('\s+', ' ', text)
  return text

In [9]:
X = data.apply(lambda x: ' '.join([' '.join(sent) for sent in rdrsegmenter.tokenize(text_preprocessing(x))]))

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2, random_state=42, stratify=label)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [11]:
def make_mask(batch_ids):
    batch_mask = []
    for ids in batch_ids:
        mask = [int(token_id > 0) for token_id in ids]
        batch_mask.append(mask)
    return torch.tensor(batch_mask)

In [12]:
def dataloader_from_text(texts=[], y=[], tokenizer=None, classes=[], savetodisk=None, loadformdisk=None, segment=False, max_len=256, batch_size=16, infer=False):
    ids_padded, masks, labels = [], [], []
    if loadformdisk == None:
        #segementer
#         if segment:
#             rdrsegmenter = VnCoreNLP("/kaggle/input/vncorenlp/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
#         texts = []
        print("LOADDING TEXT FILE")
#         with open(text_file, 'r') as f_r:
#             for sample in tqdm(f_r):
#                 if infer:
#                     text = sample.strip()
#                     if segment:
#                             text = rdrsegmenter.tokenize(text)
#                             text = ' '.join([' '.join(x) for x in text])
#                     texts.append(text)
#                 else:
#                     splits = sample.strip().split(" ",1)
#                     label = classes.index(splits[0])
#                     text = splits[1]
#                     if segment:
#                         text = rdrsegmenter.tokenize(text)
#                         text = ' '.join([' '.join(x) for x in text])
#                     labels.append(label)
#                     texts.append(text)
        for label in y:
            labels.append(classes.index(label))

        print("TEXT TO IDS")
        ids = []
        for text in tqdm(texts):
            encoded_sent = tokenizer.encode(text)
            ids.append(encoded_sent)

        del texts
        # print("PADDING IDS")
        ids_padded = pad_sequences(ids, maxlen=max_len, dtype="long", value=0, truncating="post", padding="post")
        del ids
        # print("CREATE MASK")
        # for sent in tqdm(ids_padded):
        #     masks.append(make_mask(sent))

        if savetodisk != None and not infer:
            with open(savetodisk, 'wb') as f:
                pickle.dump(ids_padded, f)
                # pickle.dump(masks, f)
                pickle.dump(labels, f)
            print("SAVED IDS DATA TO DISK")
    else:
        print("LOAD FORM DISK")
        if loadformdisk != None:
            try:
                with open(savetodisk, 'rb') as f:
                    ids_padded = pickle.load(ids_padded, f)
                    # masks = pickle.load(masks, f)
                    labels = pickle.load(labels, f)
                print("LOADED IDS DATA FORM DISK")
            except:
                print("LOAD DATA FORM DISK ERROR!")
                
    print("CONVERT TO TORCH TENSOR")
    ids_inputs = torch.tensor(ids_padded)
    del ids_padded
    # masks = torch.tensor(masks)
    if not infer:
        labels = torch.tensor(labels)

    print("CREATE DATALOADER")
    if infer:
        # input_data = TensorDataset(ids_inputs, masks)
        input_data = TensorDataset(ids_inputs)
    else:
        input_data = TensorDataset(ids_inputs, labels)
        # input_data = TensorDataset(ids_inputs, masks, labels)
    input_sampler = SequentialSampler(input_data)
    dataloader = DataLoader(input_data, sampler=input_sampler, batch_size=batch_size)

    print("len dataloader:", len(dataloader))
    print("LOAD DATA ALL DONE")
    return dataloader

In [13]:
class ROBERTAClassifier(torch.nn.Module):
    def __init__(self, num_labels, bert_model, dropout_rate=0.3):
        super(ROBERTAClassifier, self).__init__()
        if bert_model != None:
            self.roberta = bert_model
        else:
            self.roberta = RobertaModel.from_pretrained("vinai/phobert-base")
        self.d1 = torch.nn.Dropout(dropout_rate)
        self.l1 = torch.nn.Linear(768, 64)
        self.bn1 = torch.nn.LayerNorm(64)
        self.d2 = torch.nn.Dropout(dropout_rate)
        self.l2 = torch.nn.Linear(64, num_labels)
        
    def forward(self, input_ids, attention_mask):
        _, x = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x = self.d1(x)
        x = self.l1(x)
        x = self.bn1(x)
        x = torch.nn.Tanh()(x)
        x = self.d2(x)
        x = self.l2(x)
        return x 

In [14]:
class BERTClassifier(torch.nn.Module):
    def __init__(self, num_labels):
        super(BERTClassifier, self).__init__()
        bert_classifier_config = RobertaConfig.from_pretrained(
            "/kaggle/input/phobert-base-transformers/PhoBERT_base_transformers/config.json",
            from_tf=False,
            num_labels = num_labels,
            output_hidden_states=False,
            )
        print("LOAD BERT PRETRAIN MODEL")
        self.bert_classifier = RobertaForSequenceClassification.from_pretrained(
            "/kaggle/input/phobert-base-transformers/PhoBERT_base_transformers/model.bin",
            config=bert_classifier_config
            )

    def forward(self, input_ids, attention_mask, labels):
        output = self.bert_classifier(input_ids=input_ids,
                                    token_type_ids=None,
                                    attention_mask=attention_mask,
                                    labels=labels
                                    )
        return output

In [15]:
class ClassifierTrainner():
    def __init__(self, bert_model, train_dataloader, valid_dataloader, epochs=10, cuda_device='cpu', save_dir=None):

        if cuda_device == "cpu":
            self.device == torch.device("cpu")
        else:
            self.device = torch.device('cuda:{}'.format(cuda_device))

        self.model = bert_model
        if save_dir != None and os.path.exists(save_dir):
            print("Load weight from file:{}".format(save_dir))
            self.save_dir = save_dir
            epcho_checkpoint_path = glob.glob("{}/model_epoch*".format(self.save_dir))
            if len(epcho_checkpoint_path) == 0:
                print("No checkpoint found in: {}\nCheck save_dir...".format(self.save_dir))
            else:
                self.load_checkpoint(epcho_checkpoint_path)
                print("Restore weight successful from: {}".format(epcho_checkpoint_path))
        else:
            self.save_dir = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
            os.makedirs(self.save_dir)
            print("Training new model, save to: {}".format(self.save_dir))

        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.epochs = epochs
        # self.batch_size = batch_size

    def save_checkpoint(self, save_path):
        state_dict = {'model_state_dict': self.model.state_dict()}
        torch.save(state_dict, save_path)
        print(f'Model saved to ==> {save_path}')

    def load_checkpoint(self, load_path):
        state_dict = torch.load(load_path, map_location=device)
        print(f'Model restored from <== {load_path}')
        self.model.load_state_dict(state_dict['model_state_dict'])

    @staticmethod    
    def flat_accuracy(preds, labels):
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        F1_score = f1_score(pred_flat, labels_flat, average='macro')
        return accuracy_score(pred_flat, labels_flat), F1_score

    def train_classifier(self):
        self.model.to(self.device)
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)

        for epoch_i in range(0, self.epochs):
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, self.epochs))
            print('Training...')

            total_loss = 0
            self.model.train()
            train_accuracy = 0
            nb_train_steps = 0
            train_f1 = 0
            best_valid_loss = 999999
            best_eval_accuracy = 0
            for step, batch in enumerate(self.train_dataloader):
                b_input_ids = batch[0].to(self.device)
                b_input_mask = make_mask(batch[0]).to(self.device)
                b_labels = batch[1].to(self.device)

                self.model.zero_grad()
                outputs = self.model(b_input_ids, 
                                    attention_mask=b_input_mask, 
                                    labels=b_labels
                                    )
                loss = outputs[0]
                total_loss += loss.item()
                
                logits = outputs[1].detach().cpu().numpy()
                label_ids = b_labels.cpu().numpy()
                tmp_train_accuracy, tmp_train_f1 = self.flat_accuracy(logits, label_ids)
                train_accuracy += tmp_train_accuracy
                train_f1 += tmp_train_f1
                nb_train_steps += 1
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                optimizer.step()
                if step % 100 == 0:
                    print("[TRAIN] Epoch {}/{} | Batch {}/{} | Train Loss={} | Train Acc={}".format(epoch_i, self.epochs, step, len(self.train_dataloader), loss.item(), tmp_train_accuracy))
                
            avg_train_loss = total_loss / len(self.train_dataloader)
            print(" Train Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
            print(" Train F1 score: {0:.4f}".format(train_f1/nb_train_steps))
            print(" Train Loss: {0:.4f}".format(avg_train_loss))

            print("Running Validation...")
            self.model.eval()
            eval_loss, eval_accuracy = 0, 0
            nb_eval_steps, nb_eval_examples = 0, 0
            eval_f1 = 0

            for batch in self.valid_dataloader:
                b_input_mask = make_mask(batch[0]).to(self.device)
                batch = tuple(t.to(self.device) for t in batch)
                b_input_ids, b_labels = batch
                with torch.no_grad():
                    outputs = self.model(b_input_ids, 
                                        attention_mask=b_input_mask,
                                        labels=b_labels
                                        )
                    tmp_eval_loss, logits = outputs[0], outputs[1]
                    logits = logits.detach().cpu().numpy()
                    label_ids = b_labels.cpu().numpy()
                    tmp_eval_accuracy, tmp_eval_f1 = self.flat_accuracy(logits, label_ids)
                    eval_accuracy += tmp_eval_accuracy
                    eval_loss += tmp_eval_loss
                    eval_f1 += tmp_eval_f1
                    nb_eval_steps += 1

            print(" Valid Loss: {0:.4f}".format(eval_loss/nb_eval_steps))
            print(" Valid Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
            print(" Valid F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))

            if best_valid_loss > eval_loss:
                best_valid_loss = eval_loss
                best_valid_loss_path = "{}/model_best_valoss.pt".format(self.save_dir)
                self.save_checkpoint(best_valid_loss_path)
            if best_eval_accuracy > eval_accuracy:
                best_eval_accuracy = eval_accuracy
                best_eval_accuracy_path = "{}/model_best_valacc.pt".format(self.save_dir)
                self.save_checkpoint(best_eval_accuracy_path)
            
            epoch_i_path = "{}/model_epoch{}.pt".format(self.save_dir, epoch_i)
            self.save_checkpoint(epoch_i_path)
            if epoch_i != 0: os.remove("{}/model_epoch{}.pt".format(self.save_dir, epoch_i-1))

        print("Training complete!")

    def predict_dataloader(self, dataloader, classes, tokenizer):
        for batch in dataloader:
            b_input_mask = make_mask(batch[0]).to(self.device)
            batch = tuple(t.to(self.device) for t in batch)
            b_input_ids = batch[0]
            with torch.no_grad():
                outputs = self.model(b_input_ids, 
                                    attention_mask=b_input_mask,
                                    labels=None
                                    )
                logits = outputs
                logits = logits.detach().cpu().numpy()
                pred_flat = np.argmax(logits, axis=1).flatten()
                print("[PREDICT] {}:{}".format(classes[int(pred_flat)], tokenizer.decode(b_input_ids)))

    def predict_test(self, dataloader, classes, tokenizer):
        pred, label, pred_txt, label_txt = [], [], [], []
        for batch in dataloader:
            b_input_mask = make_mask(batch[0]).to(self.device)
            batch = tuple(t.to(self.device) for t in batch)
            b_input_ids, b_labels = batch
            with torch.no_grad():
                outputs = self.model(b_input_ids, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels
                                    )
                tmp_eval_loss, logits = outputs[0], outputs[1]
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.cpu().numpy()
                pred_flat = np.argmax(logits, axis=1).flatten()
                labels_flat = label_ids.flatten()
                for y in pred_flat:
                    pred.append(y)
                for lab in labels_flat:
                    label.append(lab)
        for y_pr in pred:
            pred_txt.append(classes[int(y_pr)])
        for y_lab in label:
            label_txt.append(classes[int(y_lab)])
        print(classification_report(pred_txt, label_txt))
        
    def predict_text(self, text, classes, tokenizer, max_len=256):
        ids = tokenizer.encode(text)
#         ids_padded = pad_sequences(ids, maxlen=max_len, dtype="long", value=0, truncating="post", padding="post")
        ids_padded = pad_sequence(ids)
        mask = [int(token_id > 0) for token_id in ids_padded]
        input_ids = torch.tensor(ids_padded)
        input_ids = torch.reshape(input_ids,(1,256)).to(self.device)
        intput_mask = torch.tensor(mask)
        intput_mask = torch.reshape(intput_mask,(1,256)).to(self.device)
        with torch.no_grad():
            logits = self.model(input_ids, 
                                attention_mask=intput_mask,
                                labels=None
                                )
            logits = logits[0].detach().cpu().numpy()
#             logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()
            print("[PREDICT] {}:{}".format(classes[int(pred_flat)], text))

In [16]:
def pad_sequence(input_ids, maxlen=256):
    if len(input_ids) > maxlen: 
            input_ids = input_ids[:maxlen] 
    else:
        input_ids = input_ids + [0]*(maxlen - len(input_ids))
    output_ids = np.array(input_ids)
    return output_ids

In [17]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

classes = pd.get_dummies(y_train).columns.tolist()

#     train_path = 'train.txt'
#     test_path = 'test.txt'

MAX_LEN = 256
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

train_dataloader = dataloader_from_text(texts=X_train.tolist(), y=y_train.tolist(), tokenizer=tokenizer, classes=classes, savetodisk=None, max_len=MAX_LEN, batch_size=16)
valid_dataloader = dataloader_from_text(texts=X_val.tolist(), y=y_val.tolist(), tokenizer=tokenizer, classes=classes, savetodisk=None, max_len=MAX_LEN, batch_size=16)
test_dataloader = dataloader_from_text(texts=X_test.tolist(), y=y_test.tolist(), tokenizer=tokenizer, classes=classes, savetodisk=None, max_len=MAX_LEN, batch_size=16)

#bert model
bert_classifier_model = BERTClassifier(len(classes))
#train model
bert_classifier_trainer = ClassifierTrainner(bert_model=bert_classifier_model, train_dataloader=train_dataloader, valid_dataloader=valid_dataloader, epochs=10, cuda_device="0") #cuda_device: "cpu"=cpu hoac 0=gpu0, 1=gpu1, 
bert_classifier_trainer.train_classifier()

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LOADDING TEXT FILE
TEXT TO IDS


...

CONVERT TO TORCH TENSOR
CREATE DATALOADER
len dataloader: 38
LOAD DATA ALL DONE
LOAD BERT PRETRAIN MODEL


Some weights of the model checkpoint at /kaggle/input/phobert-base-transformers/PhoBERT_base_transformers/model.bin were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model check

Training new model, save to: 12-12-2022_04-28-43


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


======== Epoch 1 / 10 ========
Training...
[TRAIN] Epoch 0/10 | Batch 0/134 | Train Loss=2.259617567062378 | Train Acc=0.0
[TRAIN] Epoch 0/10 | Batch 100/134 | Train Loss=0.5928947925567627 | Train Acc=0.875
 Train Accuracy: 0.6775
 Train F1 score: 0.6046
 Train Loss: 1.1535
Running Validation...
 Valid Loss: 0.6865
 Valid Accuracy: 0.8030
 Valid F1 score: 0.7310
Model saved to ==> 12-12-2022_04-28-43/model_best_valoss.pt
Model saved to ==> 12-12-2022_04-28-43/model_epoch0.pt
======== Epoch 2 / 10 ========
Training...
[TRAIN] Epoch 1/10 | Batch 0/134 | Train Loss=0.9056756496429443 | Train Acc=0.6875
[TRAIN] Epoch 1/10 | Batch 100/134 | Train Loss=0.18354474008083344 | Train Acc=1.0
 Train Accuracy: 0.8610
 Train F1 score: 0.8207
 Train Loss: 0.5045
Running Validation...
 Valid Loss: 0.5090
 Valid Accuracy: 0.8494
 Valid F1 score: 0.7912
Model saved to ==> 12-12-2022_04-28-43/model_best_valoss.pt
Model saved to ==> 12-12-2022_04-28-43/model_epoch1.pt
======== Epoch 3 / 10 ========
Trai

In [18]:
from sklearn.metrics import classification_report
bert_classifier_trainer.predict_test(dataloader=test_dataloader, classes=classes, tokenizer=tokenizer)

                         precision    recall  f1-score   support

                  Other       0.60      0.75      0.67        80
       du-lich_diem-den       0.98      0.88      0.93        59
      giai-tri_gioi-sao       0.96      0.91      0.94        57
       giao-duc_tin-tuc       0.86      0.89      0.88        57
       khoa-hoc_tin-tuc       0.96      0.89      0.92        55
      suc-khoe_cac-benh       0.78      0.83      0.80        46
       suc-khoe_tin-tuc       0.88      0.75      0.81        69
       the-thao_bong-da       0.87      0.84      0.85        62
  the-thao_cac-mon-khac       0.91      0.96      0.94        55
the-thao_world-cup-2022       0.85      0.80      0.82        55

               accuracy                           0.85       595
              macro avg       0.87      0.85      0.86       595
           weighted avg       0.86      0.85      0.85       595

